In [ ]:
import datetime
import numpy as np
import astropy.coordinates as coord
import astropy.time as atime
import astropy.units as u

# Astropy Time

The `astropy.time` subpackage is designed for handling precise times and dates, and for converting between standard human date/time representations and astronomical time representations. The package uses precise internal manipulations to reach sub-nanosecond timing precision. Note that this section is meant to just provide a quick overview of the basic functionality available in `astropy.time`.

Commmon astronomical time formats are julian dates (JD) and modified JD (MJD).
Let's see how to represent and convert these formats using the key object for the `time` subpackage: `Time`.

Let's start by creating a `time` using an array of MJD values:

In [ ]:
t1 = atime.Time([58083.124, 58084.25, 58085.78], format='mjd')

We can convert this to an array of Python [datetime](https://docs.python.org/3/library/datetime.html) objects using the `.datetime` attribute:

In [ ]:
t1.datetime

Or change the format to, e.g., JD:

In [ ]:
t1.jd

We can also create objects using these other formats:

In [ ]:
t2 = atime.Time(2458083.624, format='jd')
t2.mjd

In [ ]:
t3 = atime.Time(datetime.datetime(2017, 11, 29, 8, 30))
t3.mjd

A nice trick to know to create a grid of times is to add a time object to a quantity array. For example, to create a grid of 100 evenly spaced times between right now and tomorrow at the same time:

In [ ]:
time_grid = atime.Time.now() + np.linspace(0, 24, 100) * u.hour
time_grid.mjd

There is much more to the `astropy.time` module, but this is meant to serve as a quick introduction and demonstration of funamental functionality. See the [time](http://docs.astropy.org/en/latest/time/index.html) documentation for more information and feel free to ask questions!

---

# Astropy Coordinates

By "coordinates", we typically mean sky coordinates, and sometimes distance, to an astronomical source. Astropy also supports representing and transforming velocities, but we'll stick to just positional information for this tutorial - see [working with velocities in `astropy.coordinates`](http://docs.astropy.org/en/latest/coordinates/velocities.html) for more information. Note also that this section is meant to provide a quick overview of some of the functionality available in `astropy.coordinates`. For a more in depth tutorial, see the [Astropy tutorial on coordinates](https://github.com/astropy/astropy-tutorials/blob/master/tutorials/notebooks/coordinates/coordinates.ipynb) or the [`astropy.coordinates` documentation](http://docs.astropy.org/en/latest/coordinates/index.html#using-astropy-coordinates).

We typically represent sky coordinates using two spherical angular values, like right ascension (RA) and declination (Dec) for equatorial coordinates. We may also have a distance to the source, completing the 3D specification of the source's position. We may instead choose to represent the position of a source using Cartesian coordinates. Or, we may want to specify our coordinates in a different reference frame, such as the Galactic coordinate system. All of these representations, and conversions / transformations between them are supported in `astropy.coordinates`. 

The easiest way to use this functionality is through the [`SkyCoord`](http://docs.astropy.org/en/latest/api/astropy.coordinates.SkyCoord.html#astropy.coordinates.SkyCoord) object. Let's create a few sky coordinate representations as examples:

In [ ]:
c1 = coord.SkyCoord(ra=150*u.deg,
                    dec=-11*u.deg)

c2 = coord.SkyCoord(ra=150*u.deg,
                    dec=-11*u.deg,
                    distance=100*u.parsec)

By default, the input coordinates are assumed to be spherical coordinates in the equatorial (ICRS J2000) reference frame. Once we have a `SkyCoord` object, we can transform to other reference frames using the `transform_to()` method:

In [ ]:
c1.transform_to('galactic')

Or the shorthand attribute:

In [ ]:
c1.galactic

This works with or without a distance because this particular transformation is just a spherical rotation:

In [ ]:
c2.galactic

Changing the reference frame (e.g., converting to Galactic) also changes the attribute names used to access the angular components. By convenention, for example, the Galactic components are referred to as `l` (latitude) and `b` (latitude):

In [ ]:
c2.galactic.l, c2.galactic.b

If we want to create a coodinate object using Galactic coordinates, we have to use these attribute names, and specify the frame:

In [ ]:
coord.SkyCoord(l=250*u.deg,
               b=33.5*u.deg,
               distance=100*u.parsec,
               frame='galactic')

We can also create coordinate objects using string angles, or any quantities with angular units:

In [ ]:
coord.SkyCoord('0:39:15.9 0:53:17.016', 
               unit=(u.hour, u.deg))

If the string specification of the angles also contains the units, then you don't have to pass the units in (but again, the default frame assumed is the ICRS equatorial frame):

In [ ]:
coord.SkyCoord('0h39m15.9s', '0d53m17.016s')

We can change the internal _representation_ of the coordinates from spherical, to some other representation. For example, we may want to get out the position in Cylindrical or Cartesian coordinates. This usually only makes sense for a 3D position (i.e. sky position + distance):

In [ ]:
c2.represent_as('cartesian')

In [ ]:
c2.represent_as('cylindrical')

The components of the resulting object can be accessed using the displayed names above. For example, to get the cylindrical components (radius, azimuth angle, z position) as `Quantity` objects from the resulting `CylindricalRepresentation`:

In [ ]:
cyl2 = c2.represent_as('cylindrical')
print(cyl2.rho, cyl2.phi, cyl2.z)

In the above example, we used scalar quantities, but much of the power in the coordinate objects comes from the fact that they also support array quantities:

In [ ]:
c3 = coord.SkyCoord(ra=np.random.normal(150., 1., size=128) * u.deg,
                    dec=np.random.normal(0., 1., size=128) *u.deg,
                    distance=np.random.normal(100., 10., size=128) * u.parsec)
c3.galactic

There are many different astronomical coordinate reference frames in use, and we try to support all of them with `astropy.coordinates`. To see a full list of all supported frames, [see the coordinates documentation](http://docs.astropy.org/en/latest/coordinates/index.html#module-astropy.coordinates).

Some frames require more information to actually define the frame transformation. For example, a common observational coordinate system is the "altitude-azimuth" system, which is defined locally at a location on the Earth at a particular time. To transform to or from this frame, we have to also specify a time and an Earth location. As an example, let's look at the altitude of Vega from London at MJD = 57734.1231.

`SkyCoord` has a convenient method `from_name()` that queries [Simbad](http://simbad.harvard.edu/simbad/) with the specified string and automatically retrieves the coordinates for that object. Let's first use that to get the sky coordinates of Vega (note that this only works with an internet connection):

In [ ]:
vega = coord.SkyCoord.from_name('Vega')
vega

In [ ]:
# approximate location of London
loc = coord.EarthLocation(lon=0.1 * u.deg,
                          lat=51.5 * u.deg)

time = atime.Time(57734.1231, format='mjd')

altaz = coord.AltAz(location=loc, obstime=time)

In [ ]:
vega_altaz = vega.transform_to(altaz)
vega_altaz.alt

Looks like it will be ~3 degrees above the horizon!

For convenience, the `EarthLocation` class also has a method for retrieving an object from an address (using the Google maps API, so this only works if you have an internet connection):

In [ ]:
coord.EarthLocation.of_address('10 Downing St., London, GB')

This just scratches the surface of what is possible with `astropy.coordinates`. Other functionality that you may be interested in are:
* [Matching coordinates to catalogs, or cross-matching catalogs](http://docs.astropy.org/en/latest/coordinates/matchsep.html)
* [Representing and transforming velocities (e.g., proper motions)](http://docs.astropy.org/en/latest/coordinates/velocities.html)
* [Outputting or printing nicely formatted coordinate strings](http://docs.astropy.org/en/latest/coordinates/formatting.html)

## Coordinates exercises:

#### Go to the wikipedia or Simbad page for your favorite object (or if you don't have one, [here's one](https://en.wikipedia.org/wiki/Hanny's_Voorwerp)). Create a `SkyCoord` object using the coordinates you find for that object

#### Transform the coordinates above to the Galactic frame - what is its Galactic latitude (`b`) in radians?

#### What is the altitude of the andromeda galaxy (M31) right now from NYC?